## 9-9. 프로젝트 - Movielens 영화 추천 실습

---

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset `사용을 권장합니다.
- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
- 별점을 **시청횟수**로 해석해서 생각하겠습니다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

### 1) 데이터 준비와 전처리

- Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'rating'] # timestamp 항목은 불필요하여 제거
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['movie_id', 'title']  # genre 항목은 불필요하여 제거
movies = movies[using_cols]
movies.head()

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


### 2) 분석해 봅시다.

In [8]:
# 분석을 위해 데이터를 병합합니다.
df = movies.set_index("movie_id") # movie_id를 인덱스로 지정합니다.
ratings = ratings.join(df, on = "movie_id") # movie_id를 기준으로 데이터를 병합합니다.

ratings.head()

,user_id,movie_id,count,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"


In [9]:
# ratings에 있는 유니크한 영화 수
movie_unique = ratings['title'].nunique()
print('ratings에 있는 유니크한 영화 개수: {}'.format(movie_unique))

ratings에 있는 유니크한 영화 개수: 3628


In [10]:
# 유저 수 확인
user_unique = ratings['user_id'].nunique()
print('ratings에 있는 유니크한 사용자 수: {}'.format(user_unique))

ratings에 있는 유니크한 사용자 수: 6039


In [11]:
# 가장 인기 있는 영화 30개
popular_movies = ratings.groupby('title')['user_id'].count().sort_values(ascending=False).head(30)
print('가장 인기 있는 영화 30개(인기순):\n {}'.format(popular_movies))

가장 인기 있는 영화 30개(인기순):
 title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                  

- 가장 인기 있는 영화가 아메리칸뷰티이다.
- 영화가 나온 연도를 보면 데이터가 90년대 중심인 것을 알 수 있다.

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [12]:
# 인덱스와 영화명으로 서로 반환할 수 있는 딕셔너리를 생성합니다.
idx_to_movie = {i:v for i,v in zip(movies['movie_id'], movies['title'])}
movie_to_idx = {v:i for i,v in zip(movies['movie_id'], movies['title'])}

In [13]:
# 영화 전체 목록을 확인해봅시다. (3952 전체 출력되기에 주석처리했습니다.)
# for key, value in idx_to_movie.items():
#     print(key, value)

In [14]:
# 좋아하는 영화 5개를 입력합니다.
my_favorite = ['Interview with the Vampire (1994)', 'Men in Black (1997)',  'Jumanji (1995)', 'Toy Story (1995)', 'Nightmare Before Christmas, The (1993)']

In [15]:
# 좋아하는 영화의 인덱스번호를  별도의 리스트에 넣어줍니다.
my_idx = []
for i in my_favorite:
    my_idx.append(movie_to_idx[i])
my_idx

[253, 1580, 2, 1, 551]

In [16]:
# 선호 영화 정보를 모아서 DataFrame로 만듭니다. 
my_playlist = pd.DataFrame({'user_id': ['6041']*5,'movie_id':my_idx, 'title': my_favorite, 'count':[5]*5})

# 유저 id가 포함되어 있는 지 확인하고 없으면 my_playlist 데이터를 추가합니다.
if not ratings.isin({'user_id':['6041']})['user_id'].any():
    ratings = ratings.append(my_playlist)

ratings.tail(10)

,user_id,movie_id,count,title
1000203,6040,1090,3,Platoon (1986)
1000205,6040,1094,5,"Crying Game, The (1992)"
1000206,6040,562,5,Welcome to the Dollhouse (1995)
1000207,6040,1096,4,Sophie's Choice (1982)
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982)
0,6041,253,5,Interview with the Vampire (1994)
1,6041,1580,5,Men in Black (1997)
2,6041,2,5,Jumanji (1995)
3,6041,1,5,Toy Story (1995)
4,6041,551,5,"Nightmare Before Christmas, The (1993)"


In [17]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()
print('user_id:', num_user)
print('movie_id:', num_movie)

user_id: 6040
movie_id: 3628


### 4) CSR matrix를 직접 만들어 봅시다.

In [18]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie column indexing OK!!')
    ratings['title'] = temp_movie_ratings
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
0,6039,253,5,427
1,6039,1580,5,175
2,6039,2,5,513
3,6039,1,5,40


In [20]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [21]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [25]:
my, Vampire = user_to_idx['6041'], movie_to_idx['Interview with the Vampire (1994)']
my_vector, Vampire_vector = als_model.user_factors[my], als_model.item_factors[Vampire]

np.dot(my_vector, Vampire_vector)

0.33160028

- 좋아하는 영화인데 내적한 값이 0.3이 나왔습니다.
- 아마 넣은 데이터들도 적고 제가 좋아하는 영화로 넣은 영화들이 비슷하지 않아서 그런건가 싶습니다.

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
# 내가 좋아하는 영화와 인덱스를 다시 확인해 봅시다.
print(my_favorite)
print(my_idx)

['Interview with the Vampire (1994)', 'Men in Black (1997)', 'Jumanji (1995)', 'Toy Story (1995)', 'Nightmare Before Christmas, The (1993)']
[253, 1580, 2, 1, 551]


In [27]:
similar_movie = als_model.similar_items(551)
similar_movie   # (movie idx, 유사도) Tuple

[(551, 0.9999998),
 (572, 0.88532054),
 (37, 0.8770584),
 (46, 0.87480736),
 (8, 0.812341),
 (536, 0.7952853),
 (547, 0.77565265),
 (619, 0.772242),
 (528, 0.77171075),
 (609, 0.7259434)]

In [28]:
# 유사한 영화 제목을 확인해봅니다.
[idx_to_movie[i[0]] for i in similar_movie]

['Nightmare Before Christmas, The (1993)',
 'Foreign Student (1994)',
 'Across the Sea of Time (1995)',
 'How to Make an American Quilt (1995)',
 'Tom and Huck (1995)',
 'Simple Twist of Fate, A (1994)',
 'Surviving the Game (1994)',
 'Ed (1996)',
 'Scout, The (1994)',
 'Homeward Bound II: Lost in San Francisco (1996)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [29]:
user = user_to_idx['6041']

artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(50, 0.40904164),
 (4, 0.38184404),
 (107, 0.37520093),
 (458, 0.31339315),
 (33, 0.3047784),
 (322, 0.28840637),
 (545, 0.26345068),
 (670, 0.2603111),
 (92, 0.25524694),
 (250, 0.2527291),
 (330, 0.24233805),
 (1, 0.21957284),
 (10, 0.21738993),
 (1090, 0.20508385),
 (596, 0.20132549),
 (32, 0.19963528),
 (119, 0.1980936),
 (678, 0.198044),
 (475, 0.19795477),
 (1130, 0.19521464)]

In [30]:
[idx_to_movie[i[0]] for i in artist_recommended]

['Usual Suspects, The (1995)',
 'Waiting to Exhale (1995)',
 'Muppet Treasure Island (1996)',
 'Geronimo: An American Legend (1993)',
 'Wings of Courage (1995)',
 'Swimming with Sharks (1995)',
 'Harlem (1993)',
 'World of Apu, The (Apur Sansar) (1959)',
 'Mary Reilly (1996)',
 'Heavyweights (1994)',
 'Tales from the Hood (1995)',
 'Toy Story (1995)',
 'GoldenEye (1995)',
 'Platoon (1986)',
 'Pinocchio (1940)',
 'Twelve Monkeys (1995)',
 'Steal Big, Steal Little (1995)',
 'Some Folks Call It a Sling Blade (1993)',
 'In the Name of the Father (1993)',
 'Howling, The (1980)']

- 추천 받은 영화 중 좋아하는 영화는 토이스토리입니다.

---

## 루브릭 관련 고려한 점

**1. CSR matrix가 정상적으로 만들어졌다.**

- 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.

**2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.**

- 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.

**3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.**

- MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

## 회고

1️⃣ 잘한 점:
- for 문으로 전체 영화데이터를 확인해봤다.
- 인덱스로 나온 값들을 확인 할 수 있도록 영화제목으로 바꾸는 작업을 하였다.

2️⃣ 어려웠던 점:
- csr_matrix 작성에 오류가 계속 떴다. 처음엔 count 관련 에러였고 그다음은 인덱스 관련 밸류에러가 떴다.
- 아지트에 관련된 질문을 확인하고 수정해서 해결했다.
- 모델 관련해서 os.environ이 학습 내용과는 무관하다고 되어있었지만 없을 경우에는 경고 문구가 표시되었다.
-  csr_matrix 개념을 이해하기 어려웠다. 추가적으로 학습할 필요가 있다.
- 실제 추천 영화도 그렇고 벡터값이 낮게 나와서 적절하지 않은 것 같다.

3️⃣ 느낀 점:
- 이번 EXPLORATION는 약간 버거웠다.
- 코드적으로 어렵다기보단 추천시스템에 대한 이해가 되지 않은 상태에서 진행을 하려고 하니 부족함을 많이 느꼈다.
- 추천받은 영화도 잘 모르겠을 뿐더러 좋아하는 영화인지도 잘 모르겠다.
- 벡터값이 우선 현저하게 낮은데 이부분을 보완하기 위해서는 내가 좋아하는 영화를 5개 이상으로 넣어야 하지 않을까 싶다.
- 만족스럽게 작성해서 제출하지 못한 점을 반성하고, 이후 복습할 때 내가 이해하지 못하고 지나간 부분에 대해 함께 작성하면 좋을 것 같다.
- 추가적으로 장르 칼럼을 사용하지 않아서 삭제해봤다. 다음번에는 영화 장르 칼럼을 살려서 유효하게 사용하면 좋을 것 같다.